# Differences in behaviour of Imported cases - ONTARIO (Canada)

The aim of this notebook is to show how different behaviours of imported cases can affect the outcome of the epidemic in the Canadian state of Ontario. The difference in these behaviours is encompased by the different values the epsilon parameter takes. Epsilon represents the proportionality constant of the reproduction number for imported cases with respect to its analog for local ones.

In [1]:
# Import libraries
import numpy as np
import math
import branchpro
import scipy.stats
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import os
import pandas as pd


In [2]:
# Read serial interval
si_file = 'si-epinow'
path = os.path.join('../data_library/serial_interval', '{}.csv'.format(si_file))
serial_interval = pd.read_csv(path, header=None)
serial_interval = serial_interval.fillna(0)
serial_intervals = serial_interval.values.T

In [3]:
# Read Ontario data
path = os.path.join('../data_library/covid_ontario/ON.csv')
data = pd.read_csv(path)[:51]

In [4]:
time_key = 'Time'
num_timepoints = max(data[time_key])
data_times = data[time_key]

# Pad with zeros the time points where we have no information on
# the number of incidences
padded_inc_data = data.set_index(time_key).reindex(
    range(
        1, max(data_times)+1)
        ).fillna(0).reset_index()
locally_infected_cases = padded_inc_data['Incidence Number']
imported_cases = padded_inc_data['Imported Cases']

start_times = np.arange(1, num_timepoints+1, dtype=int)
times = np.arange(num_timepoints+1)

In [5]:
# Inference R_t profile, but using the LocImpBranchProPosterior
tau = 2
R_t_start = tau+1
a = 1
b = 0.8

# Transform our incidence data into pandas dataframes
inc_data = pd.DataFrame(
    {
        'Time': start_times,
        'Incidence Number': locally_infected_cases
    }
)

imported_inc_data = pd.DataFrame(
    {
        'Time': start_times,
        'Incidence Number': imported_cases
    }
)

epsilon = 0
inference = branchpro.LocImpBranchProPosteriorMultSI(
    inc_data=inc_data,
    imported_inc_data=imported_inc_data,
    epsilon=epsilon,
    daily_serial_intervals=serial_intervals,
    alpha=a,
    beta=1/b)

inference.run_inference(tau=tau)
intervals = inference.get_intervals(central_prob=.95)

new_rs = intervals['Mean'].values.tolist()

In [9]:
# Construct LocImpBranchProModel object
epsilon_values = [-0.75, 0, 1.5]

initial_r = new_rs[0]
si = np.median(serial_intervals, axis=0)
parameters = 0 # initial number of cases

all_local_cases = []
num_simulations = 100000
samples = []
central_prob = 0.95

for _, epsilon in enumerate(epsilon_values):
    m = branchpro.LocImpBranchProModel(initial_r, si, epsilon)

    m.set_r_profile(new_rs, start_times[R_t_start:])

    m.set_imported_cases(start_times, imported_cases)

    for sim in range(num_simulations):
        samples.append(m.simulate(parameters, times))
    
    simulation_estimates = np.mean(np.vstack(samples), axis=0)
    lb = 100*(1-central_prob)/2
    ub = 100*(1+central_prob)/2
    simulation_interval = np.percentile(
        np.vstack(samples), q=np.array([lb, ub]), axis=0)

    simulation_df = pd.DataFrame(
        {
            'Mean': simulation_estimates,
            'Lower bound CI': simulation_interval[0],
            'Upper bound CI': simulation_interval[1]
        }
    )

    all_local_cases.append(simulation_df)

In [11]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=imported_cases,
        name='Imported Cases'
    )
)

colours = ['green', 'indigo', 'orange']

for _, epsilon in enumerate(epsilon_values):
    fig.add_trace(
        go.Scatter(
            x=times,
            y=all_local_cases[_]['Mean'],
            name='Local Incidences for epsilon = {}'.format(epsilon),
            line_color=colours[_]
        )
    )

    fig.add_trace(
        go.Scatter(
            x=times.tolist() + times.tolist()[::-1],
            y=all_local_cases[_]['Upper bound CI'].values.tolist() + all_local_cases[_]['Lower bound CI'].values.tolist()[::-1],
            fill='toself',
            fillcolor=colours[_],
            line_color=colours[_],
            opacity=0.15,
            mode='lines',
            name='Credible interval'
        )
    )



# Add axis labels
fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)

fig.show()